# Initial Setup

In [2]:
import os

from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection"

shortcut_path = "/content/cad_detection"
if not os.path.exists(shortcut_path):
    !ln -s {project_path} {shortcut_path}
    print(f"Shortcut created at {shortcut_path}")
else:
    print(f"Shortcut at {shortcut_path} already exists.")

# Verify that the path is correct by listing its contents
print(f"\nContents of your project path: {project_path}")
!ls -l {project_path}

# Set the project path as an environment variable for easy access in shell commands
os.environ['PROJECT_PATH'] = project_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shortcut created at /content/cad_detection

Contents of your project path: /content/drive/MyDrive/PhD_Projects/cad_detection
total 4
drwx------ 2 root root 4096 Aug 26 08:52 CAD_Sparse2DGS


## Building the environment

In [4]:
# ==============================================================================
#           THE ONE-CELL SOLUTION: Build The Entire Environment
# ==============================================================================
import os
import subprocess

# --- Part 0: Define Project Path ---
# This is the actual folder containing your code, requirements.txt, submodules, etc.
project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS"
os.environ['PROJECT_PATH'] = project_path
print(f"Project path set to: {project_path}")


# --- Part 1: Setup Python 3.8 ---
print("\n--- [Part 1/5] Setting up Python 3.8 ---")
!sudo apt-get update -y > /dev/null
!sudo apt-get install -y python3.8 python3.8-distutils python3.8-dev > /dev/null
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1 > /dev/null
!curl -s https://bootstrap.pypa.io/pip/3.8/get-pip.py | python3.8 > /dev/null
PYTHON_EXECUTABLE = "/usr/bin/python3.8"
print("Python 3.8 is set up.")


# --- Part 2: Install PyTorch ---
print("\n--- [Part 2/5] Installing PyTorch ---")
!{PYTHON_EXECUTABLE} -m pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu121 > /dev/null
print("PyTorch is installed.")


# --- Part 3: Install Requirements ---
print("\n--- [Part 3/5] Installing remaining packages from requirements.txt ---")
req_file = os.path.join(project_path, "requirements.txt")
!sed -i '/^torch/s/^/#/' {req_file}
!{PYTHON_EXECUTABLE} -m pip install --ignore-installed -r {req_file} > /dev/null
print("Remaining packages are installed.")


# --- Part 4: Initialize Submodules ---
print("\n--- [Part 4/5] Initializing git submodules ---")
!cd {project_path} && git submodule update --init --recursive
print("Submodules are initialized.")


# --- Part 5: Clean and Compile Custom Modules ---
print("\n--- [Part 5/5] Cleaning and Compiling Custom Modules ---")
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.5 8.0 8.6"
submodules_path = os.path.join(project_path, "submodules")
submodules_to_install = ["diff-surfel-rasterization", "simple-knn"]

# Clean up any previous artifacts
!rm -rf {submodules_path}/*/build {submodules_path}/*/dist {submodules_path}/*/*.egg-info

for submodule in submodules_to_install:
    print(f"--- Compiling {submodule} ---")
    submodule_dir = os.path.join(submodules_path, submodule)

    try:
        # This command is designed to FAIL LOUDLY.
        # If the pip install command returns any error code, this will stop and print the error.
        result = subprocess.run(
            f'{PYTHON_EXECUTABLE} -m pip install -e .',
            cwd=submodule_dir,
            shell=True,
            capture_output=True,
            text=True,
            check=True  # This is the key: it will raise an error on failure.
        )
        print(f"✅ Successfully compiled and installed {submodule}")

    except subprocess.CalledProcessError as e:
        print(f"❌❌❌ ERROR: FAILED to compile {submodule} ❌❌❌")
        print("--- COMPILER OUTPUT (STDOUT) ---")
        print(e.stdout)
        print("\n--- COMPILER ERROR (STDERR) ---")
        print(e.stderr)
        # Break the loop on the first failure
        break

print("\n\n--- Build process finished. ---")

--- [Part 1/4] Setting up Python 3.8 (with -dev package) ---
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Python 3.8 and development headers are set up.

--- [Part 2/4] Installing a modern, compatible PyTorch ---
Looking in indexes: https://download.pytorch.org/whl/cu121
PyTorch is installed.

--- [Part 3/4] Installing remaining packages from requirements.txt ---
  Using cached gputil-1.4.0-py3-none-any.whl
  Using cached imageio-2.34.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached matplotlib-3.7.5-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.7 kB)
  Using cached mediapy-1.2.4-py3-none-any.whl.metadata (4.8 kB)
  Using cached open3d-0.18.0-cp38-cp38-manylinux_2_27_x86_64.whl.metadata (4.2 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cache

### Ground ZERO Test for modules

In [ ]:
import os
import subprocess

# --- Define All Known Paths ---
project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS"
# The editable install points to the source, so this path is critical
submodule_source_path = os.path.join(project_path, "submodules/diff-surfel-rasterization")
# The standard pip install locations
site_packages_path1 = "/usr/local/lib/python3.8/dist-packages"
site_packages_path2 = "/usr/lib/python3.8/site-packages"
PYTHON_EXECUTABLE = "/usr/bin/python3.8"

# --- Construct a Python Command to Force the Import ---
# This command string will be executed by the python3.8 interpreter.
import_command = f"""
import sys
import os

# Manually insert every possible path to be 1000% certain
sys.path.insert(0, '{project_path}')
sys.path.insert(0, '{submodule_source_path}')
sys.path.insert(0, '{site_packages_path1}')
sys.path.insert(0, '{site_packages_path2}')

print('--- Python Search Path (sys.path) ---')
import pprint
pprint.pprint(sys.path)
print('------------------------------------')

try:
    print('\\n--- Attempting to import diff_surfel_rasterization ---')
    from diff_surfel_rasterization import GaussianRasterizationSettings, GaussianRasterizer
    print('✅✅✅ SUCCESS: The module was imported correctly! ✅✅✅')
except Exception as e:
    print('❌❌❌ FAILED: The import failed. ❌❌❌')
    # --- THIS LINE IS NOW FIXED ---
    print('The error was: ' + repr(e)) # Using repr() to avoid the f-string bug
    import traceback
    traceback.print_exc()
"""

# --- Execute the Command with Subprocess ---
print("--- EXECUTING GROUND ZERO TEST (Corrected) ---")
result = subprocess.run(
    [PYTHON_EXECUTABLE, "-c", import_command],
    capture_output=True,
    text=True
)

print("\n--- SUBPROCESS STDOUT ---")
print(result.stdout)
print("\n--- SUBPROCESS STDERR ---")
print(result.stderr)

## Github Access

### Setup

In [ ]:
# ==============================================================================
# CELL 1: Create .gitignore (Run Once for the Project)
# ==============================================================================
import os

# This content tells Git to ignore temporary files, build artifacts, and datasets.
gitignore_content = """
# Python standard ignores
__pycache__/
*.pyc

# Build artifacts from our manual compilation
**/build/
**/dist/
**/*.egg-info/

# Jupyter/Colab notebook checkpoints
.ipynb_checkpoints/

# Downloaded ZIP files
*.zip

# IMPORTANT: Add your dataset folders here if you have any!
# For example:
# dataset/
# DTU_dataset/
"""

project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS"
gitignore_path = os.path.join(project_path, ".gitignore")

# This writes the content to the file
with open(gitignore_path, "w") as f:
    f.write(gitignore_content)

print(f"✅ .gitignore file created successfully at: {gitignore_path}")

In [ ]:
# ==============================================================================
# CELL 2: GIT SETUP (RUN ONCE PER SESSION)
# ==============================================================================
import os
from google.colab import userdata

# --- 1. Configuration (Check that these are correct) ---
project_path = "/content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS"
github_username = "alirezafalah"
your_name = "Alireza Falah"
your_email = "al.r.falah@gmail.com"
repo_name = "CAD_Sparse2DGS"

# --- 2. Execution ---
print(f"Navigating to: {project_path}")
os.chdir(project_path)

print("\nConfiguring Git user for this session...")
!git config --global user.name "{your_name}"
!git config --global user.email "{your_email}"

print("Preparing authenticated connection to GitHub...")
try:
    github_token = userdata.get('GITHUB_TOKEN')
    remote_url = f"https://{github_token}@github.com/{github_username}/{repo_name}.git"
    print("\n✅ Setup complete. You can now use the 'Commit & Push' cell.")
except userdata.SecretNotFoundError:
    print("\n\n❌ ERROR: Secret 'GITHUB_TOKEN' not found. Please add it to the secret manager (🔑 icon).")
except Exception as e:
    print(f"\n\n❌ An error occurred during setup: {e}")

Navigating to: /content/drive/MyDrive/PhD_Projects/cad_detection/CAD_Sparse2DGS

Configuring Git user for this session...
Preparing authenticated connection to GitHub...

✅ Setup complete. You can now use the 'Commit & Push' cell.


### Commit

In [ ]:
# ==============================================================================
# CELL 3: COMMIT & PUSH (RUN AS NEEDED)
# ==============================================================================

#@markdown ### Fill out the details for this commit:
#@markdown Type the filenames you want to commit, separated by spaces.
files_to_commit = "2DGS_test.ipynb requirements.txt .gitignore" #@param {type:"string"}
#@markdown Type a descriptive message for your commit.
commit_message = "feat: Finalize setup, fix all dependencies, and add gitignore" #@param {type:"string"}

# --- Execution ---
print("--- Checking Git Status ---")
!git status

print(f"\n--- Staging files: {files_to_commit} ---")
!git add {files_to_commit}

print(f"\n--- Committing with message: '{commit_message}' ---")
!git commit -m "{commit_message}"

print("\n--- Pushing changes to GitHub ---")
# The 'remote_url' variable was created in the setup cell.
!git push {remote_url} main

print("\n\n✅ Push complete!")

--- Checking Git Status ---
^C

--- Staging files: 2DGS_test.ipynb requirements.txt .gitignore ---

--- Committing with message: 'feat: Finalize setup, fix all dependencies, and add gitignore' ---
[main 307c776] feat: Finalize setup, fix all dependencies, and add gitignore
 3 files changed, 26 insertions(+), 5 deletions(-)
 create mode 100644 .gitignore
 create mode 100644 2DGS_test.ipynb

--- Pushing changes to GitHub ---
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 14.32 KiB | 666.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/alirezafalah/CAD_Sparse2DGS.git
   af15587..307c776  main -> main


✅ Push complete!


# Running


In [21]:
project_path = "cad_detection/CAD_Sparse2DGS"
os.chdir(project_path)

2DGS_test.ipynb  data		    lpipsPyTorch  render.py	    train.py
arguments	 debug_env.py	    metrics.py	  requirements.txt  utils
assets		 exp_dtu	    models	  scene		    view.py
CAD_Sparse2DGS	 gaussian_renderer  MVS		  scripts
convert.py	 get-pip.py	    README.md	  submodules


In [ ]:
!python3.8 ./scripts/train_all.py

Starting training process for all scenes...

----- Processing Scene: 24 -----
Executing command:
PYTHONPATH=/usr/lib/python3.8/site-packages:$PYTHONPATH OMP_NUM_THREADS=1 CUDA_VISIBLE_DEVICES=0 python3.8 train.py -s data/dtu_sparse/dtu_sparse/scan24 -m exp_dtu/release/scan24 -r 2 --depth_ratio 1 --lambda_dist 1000 --port 1324
Optimizing exp_dtu/release/scan24
Output folder: exp_dtu/release/scan24 [04/09 15:52:13]
Tensorboard not available: not logging progress [04/09 15:52:13]
Reading camera 3/3 [04/09 15:52:17]
Loading Training Cameras [04/09 15:52:18]
Loading Test Cameras [04/09 15:52:21]
successfully building CLMVSNet model ! [04/09 15:52:22]
load pretrained model from models/model_clmvsnet.ckpt [04/09 15:52:28]
cal 0023 mvs depth [04/09 15:52:29]
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)